In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sklearn import neighbors
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'diabetes'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

diabetes_df = pd.read_sql_query('select * from diabetes',con=engine)

engine.dispose()

diabetes_df.info()
diabetes_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763 entries, 0 to 762
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pregnancies        763 non-null    int64  
 1   glucose            763 non-null    int64  
 2   blood_pressure     763 non-null    int64  
 3   skin_thickness     763 non-null    int64  
 4   insulin            763 non-null    int64  
 5   bmi                763 non-null    float64
 6   diabetes_pedigree  763 non-null    float64
 7   age                763 non-null    int64  
 8   outcome            763 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 53.8 KB


,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree,age,outcome
0,5,44,62,0,0,25.0,0.587,36,0
1,2,56,56,28,45,24.2,0.332,22,0
2,9,57,80,37,0,32.8,0.096,41,0
3,0,57,60,0,0,21.7,0.735,67,0
4,3,61,82,28,0,34.4,0.243,46,0


In [2]:
#Target variable
Y = diabetes_df['glucose']

#feature set
X = diabetes_df[['bmi','blood_pressure', 'insulin']]

In [3]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [4]:
#Gridsearch for best parameters

In [5]:
param = {'n_neighbors': range(0,7), 'weights':['uniform','distance']}

neighbors = KNeighborsClassifier(n_neighbors=1)
grd = GridSearchCV(neighbors, param, n_jobs=-1)
grd.fit(X, Y)

GridSearchCV(estimator=KNeighborsClassifier(n_neighbors=1), n_jobs=-1,
             param_grid={'n_neighbors': range(0, 7),
                         'weights': ['uniform', 'distance']})

In [6]:
grd.best_params_

{'n_neighbors': 6, 'weights': 'uniform'}

#### OLS Regression Model

In [7]:
#add constant
X_train_c = sm.add_constant(X_train)

#Fit OLS model
results = sm.OLS(y_train,X_train_c).fit()

#print
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                glucose   R-squared:                       0.153
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     34.23
Date:                Mon, 17 Aug 2020   Prob (F-statistic):           2.40e-20
Time:                        21:49:33   Log-Likelihood:                -2713.0
No. Observations:                 572   AIC:                             5434.
Df Residuals:                     568   BIC:                             5451.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             86.4402      5.735     15.

In [8]:
y_predict_test = results.predict(sm.add_constant(X_test))

In [9]:
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_predict_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_predict_test) / y_test)) * 100))

Root mean squared error of the prediction is: 29.36520005518959
Mean absolute percentage error of the prediction is: 20.35150806470403


#### KNN Regression

In [14]:
knn = KNeighborsClassifier(n_neighbors=6, weights='uniform')

#fitting the model
knn.fit(X_train,y_train)

#predict
knn.predict(X_test)

#cross validation
from sklearn.model_selection import cross_val_score
score = cross_val_score(knn, X, Y, cv=6)
print(score)


[0.015625   0.00787402 0.04724409 0.02362205 0.01574803 0.02362205]
